**imports:**

In [12]:
import numpy as np

**main functions:**

In [2]:
def calc_TFL_dist(prev_container, curr_container, focal, pp):
    norm_prev_pts, norm_curr_pts, R, foe, tZ = prepare_3D_data(prev_container, curr_container, focal, pp)
    if abs(tZ) < 10e-6:
        print('tz = ', tZ)
    elif norm_prev_pts.size == 0:
        print('no prev points')
    elif norm_curr_pts.size == 0:
        print('no curr points')
    else:
        curr_container.corresponding_ind, curr_container.traffic_lights_3d_location, curr_container.valid = calc_3D_data(
            norm_prev_pts, norm_curr_pts, R, foe, tZ)
    return curr_container


def prepare_3D_data(prev_container, curr_container, focal, pp):
    norm_prev_pts = normalize(prev_container.traffic_light, focal, pp)
    norm_curr_pts = normalize(curr_container.traffic_light, focal, pp)
    R, foe, tZ = decompose(np.array(curr_container.EM))
    return norm_prev_pts, norm_curr_pts, R, foe, tZ


def calc_3D_data(norm_prev_pts, norm_curr_pts, R, foe, tZ):
    norm_rot_pts = rotate(norm_prev_pts, R)
    pts_3D = []
    corresponding_ind = []
    valid_vec = []
    for p_curr in norm_curr_pts:
        corresponding_p_ind, corresponding_p_rot = find_corresponding_points(p_curr, norm_rot_pts, foe)
        Z = calc_dist(p_curr, corresponding_p_rot, foe, tZ)
        valid = (Z > 0)
        if not valid:
            Z = 0
        valid_vec.append(valid)
        P = Z * np.array([p_curr[0], p_curr[1], 1])
        pts_3D.append((P[0], P[1], P[2]))
        corresponding_ind.append(corresponding_p_ind)
    return corresponding_ind, np.array(pts_3D), valid_vec

**help functions:**

In [3]:
def normalize(pts, focal, pp):
    # transform pixels into normalized pixels using the focal length and principle point
    return np.array([[(pixel[0] - pp[0]) / focal, (pixel[1] - pp[1]) / focal, 1] for pixel in pts])


def unnormalize(pts, focal, pp):
    # transform normalized pixels into pixels using the focal length and principle point
    return np.array([[(pixel[0] * focal) + pp[0], (pixel[1] * focal) + pp[1], focal] for pixel in pts])


def decompose(EM):
    # extract R, foe and tZ from the Ego Motion

    t = EM[:3, 3]
    return EM[:3, :3], [t[0] / t[2], t[1] / t[2]], t[2]


def rotate(pts, R):
    # rotate the points - pts using R

    return np.array([(R @ pixel) for pixel in pts])


def find_corresponding_points(p, norm_pts_rot, foe):
    # compute the epipolar line between p and foe
    # run over all norm_pts_rot and find the one closest to the epipolar line
    # return the closest point and its index

    m = (foe[1] - p[1]) / (foe[0] - p[0])
    n = ((p[1] * foe[0]) - (foe[1] * p[0])) / (foe[0] - p[0])
  #if the car dont move
    #  Distance of point from line 𝑦=𝑚𝑥+𝑛 calculate by the formula: |(𝑚𝑥+𝑛−𝑦)/√(𝑚^2+1)|
    dis = [abs((m * pixel[0] + n - pixel[1]) / (m ** 2 + 1) ** 0.5) for pixel in norm_pts_rot]

    return np.argmin(dis), norm_pts_rot[np.argmin(dis)]


def calc_dist(p_curr, p_rot, foe, tZ):
    # calculate the distance of p_curr using x_curr, prev_x_rotate, foe_x and tZ from me
    # calculate the distance of p_curr using y_curr, y_rot, foe_y and tZ
    # combine the two estimations and return estimated Z

    zX = (tZ * (foe[0] - p_rot[0])) / (p_curr[0] - p_rot[0])
    zY = (tZ * (foe[1] - p_rot[1])) / (p_curr[1] - p_rot[1])

    dx_plus_dy = (p_rot[0] - p_curr[0]) + (p_rot[1] - p_curr[1])
    return np.average([zX, zY],
                      weights=[(p_rot[0] - p_curr[0]) / dx_plus_dy, (p_rot[1] - p_curr[1]) / dx_plus_dy])
